In [3]:
import csv
import pandas as pd
import random
import datetime
# need to generate for 
# Booking (FK Client)
# FlightSchedule (FK Route, Airline) 
# Flight (FK FlightSchedule) 
# Seat (FK Flight)
# Ticket (FK Booking, Passenger, Seat)

In [39]:
# Booking has :
# confirmationNo (PK)
# bookingStatus, 
# numSeatsBooked,
# numPointsEarned, 
# transactionDate, 
# transactionAmount,
# username (FK Client)

In [58]:
df = pd.read_csv('Client.csv', header=None)

# extract username col (100 rows)
df = df[0]

# possible booking status
status = ['Booked', 'Completed', 'Cancelled']

# range for random date
start_date = datetime.date(2020, 1, 1)
end_date = datetime.date(2020, 11, 24)
time_between_dates = end_date - start_date
days_between_dates = time_between_dates.days

# create an empty df
booking_df = pd.DataFrame(columns=['confirmationNo','bookingStatus','numSeatsBooked','numPointsEarned','transactionDate','transactionAmount','username'])
for i in range(10000):
    status_index = random.randint(0,2)
    user_index = random.randint(0,99)
    num_seats = random.randint(1,5)
    random_number_of_days = random.randrange(days_between_dates)
    random_date = start_date + datetime.timedelta(days=random_number_of_days)
    amount = num_seats * random.randint(100,1000)
    points = int(amount * 0.1)
    booking_df.loc[i]=[i+1,status[status_index],num_seats, points, random_date, amount, df[user_index]]

In [60]:
booking_df.to_csv('bookings.csv', index = False)

In [104]:
# FlightSchedule has:
# flightNo (PK)
# departureAirportIATA (FK Route)
# arrivalAirportIATA (FK Route)
# airlineIATA (FK Airline)
# departureWeekday
# departureTime
# arrivalWeekday
# arrivalTime
# seatPrice 

In [161]:
route = pd.read_csv('routes.csv', header=None)
route = route.drop(columns=2) # drop distance
route = route.drop_duplicates(subset = [0,1])

# for the purpose of this project, I will limit to routes that departure from Pearson Airport only (YYZ)
departure = 'YYZ'
arrival = pd.DataFrame(route.loc[route[0] == departure][1],index=None)
arrival.reset_index(inplace = True) 
arrival = arrival[1] # 147 rows

airline = pd.read_csv('airlines.csv', header=None)
airline = airline.drop(columns=1) 
airline = airline[0] #943 rows

weekdays = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

flightSchedule_df = pd.DataFrame(columns=['flightNo','departureAirportIATA','arrivalAirportIATA','airlineIATA',
                                          'departureWeekday','departureTime','arrivalWeekday', 'arrivalTime', 'seatPrice'])
for i in range(1000):
    weekday_index = random.randint(0,6)
    airline_index = random.randint(0,942)
    airport_index = random.randint(0,146)
    depart_hour = random.randint(0,12)
    depart_minute = random.randint(0,59)
    arrival_hour = random.randint(13,23)
    arrival_minute = random.randint(0,59)
    seat_price = random.randint(100,1000)
    flightSchedule_df.loc[i] = [str(airline[airline_index])+str(i), 'YYZ', arrival[airport_index], airline[airline_index],
                               weekdays[weekday_index], str(depart_hour)+':'+str(depart_minute), weekdays[weekday_index],
                               str(arrival_hour)+':'+str(arrival_minute), seat_price]

In [163]:
flightSchedule_df.to_csv('flightSchedules.csv', index = False)

In [ ]:
# Flight has:
# flightNo (PK, FK FlightSchedule)
# departureDate (PK)
# arrivalDate
# totalNumSeats
# numSeatsSold

In [213]:
schedule = pd.read_csv('flightSchedules.csv')
flightNo = schedule['flightNo'] 
departureWeekday = schedule['departureWeekday'] # 1000 rows

# a dictonary used to store date and weekday pair
all_dates = {}
# for every day in our range, append the date and its weekday into the dictionary
for i in range(days_between_dates+1):
    date = start_date + datetime.timedelta(days = i)
    weekday = weekdays[date.weekday()]
    all_dates[date.strftime('%Y-%m-%d')] = weekday

flight_df = pd.DataFrame(columns = ['flightNo', 'departureDate', 'arrivalDate', 'totalNumSeats', 'numSeatsSold'])

index = 0
for i in range(1000):
    # num seats for this flightNo should be fixed
    num_seats = random.randint(200,1000)
    for date, weekday in all_dates.items():
        # insert a new flight for this flight schedule on the date that fits the schedule
        if departureWeekday[i] == weekday:
            # generate a random sold number that is less than or equal to total seats
            seats_sold = num_seats - random.randint(0,num_seats)
            flight_df.loc[index] = [flightNo[i], date, date, num_seats, seats_sold]
            index += 1


In [215]:
flight_df.to_csv('flights.csv', index = False)

In [4]:
# Seat has:
# seatNo (PK)
# seatStatus
# flightNo (PK, FK Flight)
# departureDate (PK, FK Flight)

In [7]:
flight = pd.read_csv('flights.csv')
flightNo = flight['flightNo']
departureDate = flight['departureDate'] #47000 rows
# because we have many flights, I will only draw 1000 random samples from those and generate 18 seats each
samples = random.sample(range(0,46999), 46999)
index = 0
seat_df = pd.DataFrame(columns=['seatNo', 'seatStatus', 'flightNo', 'departureDate'])
status = ['Available', 'Sold']
# for each flight that was drawn
for i in samples:
    for j in range(1,2):
        for x in ['A', 'B']:
            seat_df.loc[index] = [str(j)+x, status[random.randint(0,1)], flightNo[i], departureDate[i]]
            index += 1;

In [8]:
seat_df.to_csv('seats_new.csv',index=False)

In [ ]:
# Ticket has:
# ticketNo (PK)
# confirmationNo (FK Booking)
# travelDocument (FK Passenger)
# seatNo (FK Seat)
# flightNo (FK Seat)
# departureDate (FK Seat)

In [270]:
ticket_df = pd.DataFrame(columns=['ticketNo','confirmationNo','travelDocument','seatNo','flightNo','departureDate'])
confirmationNo = pd.read_csv('bookings.csv')['confirmationNo'] # 10000 rows
travelDocument = pd.read_csv('Passenger.csv')['travelDocument'] # 100 rows
seatNo = seat_df['seatNo']
flightNo = seat_df['flightNo']
departureDate = seat_df['departureDate'] #18000 rows
index = 0

for i in range(1000):
    # for each flight, make sure 18 different people are on flight
    travelDoc_index = random.sample(range(100), 18)
    for j in range(9):
        ticket_df.loc[index] = [index+1, confirmationNo[i*9+j], travelDocument[travelDoc_index[j]],
                               seatNo[index], flightNo[index], departureDate[index]]
        index += 1
        ticket_df.loc[index] = [index+1, confirmationNo[i*9+j], travelDocument[travelDoc_index[j+9]],
                               seatNo[index], flightNo[index], departureDate[index]]
        index += 1

In [272]:
ticket_df.to_csv('tickets.csv', index=False)